# Semantic Memory


In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# pip install langchain langchain-openai langchain-anthropic langchain-google-vertexai
from langchain.chat_models import init_chat_model

checkpointer = InMemorySaver()
store = InMemoryStore()

In [ ]:
llm_gpt_5 = init_chat_model("openai:gpt-5")

In [ ]:
agent = create_react_agent(llm_gpt_5, tools=[], store=store, checkpointer=checkpointer)

In [ ]:
def chat(agent, txt, thread_id):
    result_state = agent.invoke(
        {"messages": [{"role": "user", "content": txt}]},
        config={"configurable": {"thread_id": thread_id}},
    )
    return result_state["messages"][-1].content

In [ ]:
thread_1 = "thread-1"
chat(
    agent,
    "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?",
    thread_1,
)

In [ ]:
chat(agent, "Nice! Wish me luck!", thread_1)

In [ ]:
thread_2 = "thread-2"
chat(
    agent,
    "Remember what I'm supposed to do for my training this week? It's week 3...",
    thread_2,
)

## Adding Memory

We'll use regular tools.


In [26]:
from langmem import create_manage_memory_tool, create_search_memory_tool

from pydantic import BaseModel

class PreferenceMemory(BaseModel):
    """Store preferences about the user."""
    category: str
    preferences: list[str]
    context: str
    recent_memories: list[str] = []


class UserProfile(BaseModel):
    name: str
    age: int | None = None
    preferences: dict | None = None   
    recent_memories: list[str] = []


store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small"
    }
)

INSTRUCTIONS = (
    "Proactively call this tool when you:\n\n"
    "1. Identify a new USER preference.\n"
    "2. Receive an explicit USER request to remember something or otherwise alter your behavior.\n"
    "3. Are working and want to record important context.\n"
    "4. Identify that an existing MEMORY is incorrect or outdated.\n"
    "5. Always store recent memories proactively."
)

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(instructions=INSTRUCTIONS, namespace=namespace, schema=PreferenceMemory, actions_permitted=("create", "update", "delete")),
    create_search_memory_tool(namespace)
]
checkpointer = InMemorySaver()
agent = create_react_agent(llm_gpt_5, tools=memory_tools, store=store, checkpointer=checkpointer, debug=True)

In [27]:
thread_1 = "thread-1"
chat(
    agent,
    "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?",
    thread_1,
)

'Great goal! Here’s a practical 8‑week, daily-structured plan to get you ready. It uses effort-based paces so it fits most fitness levels. If you share your current weekly mileage, longest recent run, and any goal time, I can fine-tune distances and paces.\n\nHow to use this plan\n- Intensity guide: \n  - Easy = conversational (RPE 3–4).\n  - Tempo = comfortably hard, around 10K–half pace (RPE 6–7).\n  - Intervals = 5K pace (RPE 7–8).\n  - Long run = easy to steady (RPE 4–5).\n- Warm-up/cooldown: 10–15 min easy jog + drills before quality days; 10 min easy after.\n- Strength: 20–30 min 2x/week (glutes, calves, hamstrings, core).\n- Fueling: On long runs ≥75 min, practice gels every 30–40 min + fluids/electrolytes.\n\nWeekly structure (same each week)\n- Mon: Rest + strength/mobility 20–30 min\n- Tue: Quality workout 1\n- Wed: Easy run\n- Thu: Quality workout 2\n- Fri: Rest or 30–45 min easy cross-train + mobility\n- Sat: Easy run\n- Sun: Long run\n\nMileage guidance\n- Choose the low e

In [28]:
print(
    chat(
        agent,
        "Nice! Wish me luck! Please note down the detailed memories for me :)",
        thread_1,
    )
)

You’ve got this—good luck! I’ve saved your detailed training framework and weekly structure so we can reference and adjust as you go. If anything changes (schedule, goal time, niggles), tell me and I’ll update the plan.


In [29]:
# Retrieve and display stored memories
print("\nStored memories:")
memories = store.search(("agent_memories",))
for memory in memories:
    print(f"\nMemory {memory.key}:")
    print(f"Content: {memory.value['content']}")
    print(f"Preferences: {memory.value['content']['preferences']}")
    print(f"Recent: {memory.value['content']['recent_memories']}")


Stored memories:

Memory 584366c8-cd97-4d6b-9b04-991efd7b6780:
Content: {'category': 'Fitness', 'preferences': ['Training for a half marathon in ~8 weeks', 'Prefers a daily-structured 8-week plan', 'Weekly structure: Mon rest+strength; Tue quality workout; Wed easy run; Thu quality workout; Fri rest or easy cross-train; Sat easy run; Sun long run', 'Effort-based pacing (RPE): Easy 3–4; Tempo 6–7; Intervals 7–8; Long run 4–5', 'Strength training 2x/week (20–30 min: glutes, calves, hamstrings, core)', 'Fueling on long runs ≥75 min: gels every 30–40 min + fluids/electrolytes', 'Taper in weeks 7–8; race on Sun of week 8'], 'context': 'Half marathon prep plan created today; target race at end of week 8. Use this structure to guide weekly check-ins and adjustments.', 'recent_memories': ['Saved 8-week HM training framework and weekly structure.', 'Week 1 key sessions: Tue 6x400m @5K pace; Thu tempo 2 mi; Sun long 6–8 mi.']}
Preferences: ['Training for a half marathon in ~8 weeks', 'Prefers a

In [30]:
thread_2 = "thread-2"
chat(
    agent,
    "Remember what I'm supposed to do for my training this week? It's week 3...",
    thread_2,
)

'Here’s your Week 3 plan based on your half marathon schedule:\n\n- Mon: Rest + 20–30 min strength (glutes, calves, hamstrings, core)\n- Tue: Intervals – 5 x 800 m @ 10K effort (RPE 7–8). Warm up 10–15 min easy + drills/strides; 400 m jog (or 2–3 min easy) between reps; cool down 10 min\n- Wed: Easy run 3–5 miles @ RPE 3–4 (optional 4 x 20-sec strides)\n- Thu: Tempo – 3 miles continuous @ RPE 6–7. Warm up 10–15 min easy; cool down 10 min\n- Fri: Rest or easy cross-train 30–45 min + mobility\n- Sat: Easy run 3–5 miles @ RPE 3–4\n- Sun: Long run 8–9 miles @ RPE 4–5; take a gel every 30–40 min and fluids/electrolytes. Optional last 1–2 miles steady if you feel good\n\nIf you’re feeling cooked, trim volume by 10–20%. Want me to convert any of this to time-based or estimate paces from a recent 5K/10K?'

In [ ]:
chat(
    agent,
    "That may be tricky. I just sprained my ankle. Could you update my plan to include more cross training? Be sure to update the existing key of our plan",
    thread_2,
)

## Different User Memories


In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool

store = InMemoryStore(index={"dims": 1536, "embed": "openai:text-embedding-3-small"})

namespace = ("agent_memories", "{user_id}")
memory_tools = [
    create_manage_memory_tool(
        namespace, actions_permitted=("create", "update", "delete")
    ),
    create_search_memory_tool(namespace),
]
checkpointer = InMemorySaver()

agent = create_react_agent(
    llm_gpt_5, tools=memory_tools, store=store, checkpointer=checkpointer
)

In [ ]:
def chat(agent, txt, thread_id, user_id):
    result_state = agent.invoke(
        {"messages": [{"role": "user", "content": txt}]},
        config={"configurable": {"thread_id": thread_id, "user_id": user_id}},
    )
    return result_state["messages"][-1].content

In [ ]:
thread_1 = "thread-1"
user_id = "User-A"
chat(
    agent,
    "Hi I'm Will, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare and help me stay honest??",
    thread_1,
    user_id,
)

In [ ]:
thread_1 = "thread-2"
user_id2 = "User-B"
chat(
    agent,
    "Hi I'm John, I'm learning chess - could you help me become great??",
    thread_1,
    user_id2,
)

In [ ]:
chat(agent, "Do you remember me liking any sports?", thread_1, user_id2)

In [ ]:
items = store.search(("agent_memories",))
for item in items:
    print(item.namespace, item.value)

## "Eager" memory retrieval

We can fetch memories before the first LLM call to simplify its response. Otherwise, it has known and unknown unknowns so will almost always try to search for some subclass of questions.


In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.config import get_store

store = InMemoryStore(index={"dims": 1536, "embed": "openai:text-embedding-3-small"})

namespace = ("agent_memories",)
memory_tools = [
    create_manage_memory_tool(
        namespace, actions_permitted=("create", "update", "delete")
    ),
    create_search_memory_tool(namespace),
]
checkpointer = InMemorySaver()


def prompt(state):
    # Search over memories based on the messages
    store = get_store()
    items = store.search(namespace, query=state["messages"][-1].content)
    memories = "\n\n".join(str(item) for item in items)
    system_msg = {"role": "system", "content": f"## Memories:\n\n{memories}"}
    return [system_msg] + state["messages"]


agent = create_react_agent(
    llm_gpt_5, prompt=prompt, tools=memory_tools, store=store, checkpointer=checkpointer
)

In [ ]:
thread_1 = "thread-1"
chat(
    agent,
    "Hi there, I'm training for a half marathon in 2 months - could you propose a daily training plan to prepare?",
    thread_1,
    None,
)

In [ ]:
print(
    chat(
        agent,
        "Nice! Wish me luck! Please note down the detailed memories for me :)",
        thread_1,
        None,
    )
)

In [ ]:
thread_2 = "thread-2"
chat(
    agent,
    "What I'm supposed to do for my training this week? It's week 3...",
    thread_2,
    None,
)